#### Génération du graph "communes" dans Neo4J

Notebook amorcé par : Jesshuan

Objectifs :

- Récupération du graph sous forme edges et nodes depuis les notebooks de mika... (fichier finalement transféré à l'a'ncienne)

- Génération dans une base de données locale Neo4j

Imports :

In [1]:
import geopandas as gpd
import pandas as pd
import geopy.distance
import os
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import copy
import joblib

from neo4j import GraphDatabase, basic_auth

In [2]:
PASSWORD_NEO4J = 'passwordneo4j'

--------

In [3]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = False

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    
import os
os.environ['USE_PYGEOS'] = '0'
%load_ext autoreload
%autoreload



%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    # 'Corse-du-Sud': '2A',
    # 'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
AREA

['Ain',
 'Allier',
 'Ardèche',
 'Cantal',
 'Drôme',
 'Isère',
 'Loire',
 'Haute-Loire',
 'Puy-de-Dôme',
 'Rhône',
 'Savoie',
 'Haute-Savoie',
 "Côte-d'Or",
 'Doubs',
 'Jura',
 'Nièvre',
 'Haute-Saône',
 'Saône-et-Loire',
 'Yonne',
 'Territoire de Belfort',
 "Côtes-d'Armor",
 'Finistère',
 'Ille-et-Vilaine',
 'Morbihan',
 'Cher',
 'Eure-et-Loir',
 'Indre',
 'Indre-et-Loire',
 'Loir-et-Cher',
 'Loiret',
 'Ardennes',
 'Aube',
 'Marne',
 'Haute-Marne',
 'Meurthe-et-Moselle',
 'Meuse',
 'Moselle',
 'Bas-Rhin',
 'Haut-Rhin',
 'Vosges',
 'Aisne',
 'Nord',
 'Oise',
 'Pas-de-Calais',
 'Somme',
 'Paris',
 'Seine-et-Marne',
 'Yvelines',
 'Essonne',
 'Hauts-de-Seine',
 'Seine-Saint-Denis',
 'Val-de-Marne',
 "Val-d'Oise",
 'Calvados',
 'Eure',
 'Manche',
 'Orne',
 'Seine-Maritime',
 'Charente',
 'Charente-Maritime',
 'Corrèze',
 'Creuse',
 'Dordogne',
 'Gironde',
 'Landes',
 'Lot-et-Garonne',
 'Pyrénées-Atlantiques',
 'Deux-Sèvres',
 'Vienne',
 'Haute-Vienne',
 'Ariège',
 'Aude',
 'Aveyron',
 'Gard

In [4]:
nodes = gpd.read_feather(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes_communes.feather'))

In [25]:
nodes_reset = nodes.reset_index()

In [26]:
nodes_reset

,osmid,y,x,codgeo,area,sleeping,visit,working,fooding,vehicule,geometry
0,1149451,50.582893,2.143174,62053,sw}KqgzsHtFrFwWhPzQtg@vn@lj@vd@jVx^p_@deAzeA`I...,3.0,1.0,NaN,NaN,NaN,POINT (2.14317 50.58289)
1,910704,49.717236,3.012390,80284,wwsQaxhnHqLqv@|Aea@`]md@zGLr`@k\hXiJvCaOcCs|@l...,NaN,1.0,NaN,NaN,NaN,POINT (3.01239 49.71724)
2,1452239,42.876728,0.610023,31290,scwB_ptdGiNs]tPyd@|e@aEt~@zEvrAdBp`@YnRbMuF|W}...,NaN,NaN,NaN,NaN,NaN,POINT (0.61002 42.87673)
3,2173628,48.094594,1.514000,28400,grkHusqdHvZcW`]_^z_AbWpLvXfQtDlK}Mnf@|O{GfL|Nv...,NaN,NaN,NaN,NaN,NaN,POINT (1.51400 48.09459)
4,106436,47.193507,0.099943,37220,oaZqd~~GntAaKgQoh@zGgFthAwXl`Aml@lv@iRpLnOgGzM...,NaN,2.0,NaN,NaN,NaN,POINT (0.09994 47.19351)
...,...,...,...,...,...,...,...,...,...,...,...
34455,2893288,46.506657,3.092677,03170,kpyQaka{Gbt@lCvJyZn\}W~KfFvWfe@}Ad_@sSlf@qp@l`...,NaN,2.0,2.0,NaN,NaN,POINT (3.09268 46.50666)
34456,186959,49.015889,-0.162877,14394,hhe@mmfjHtCxd@mmA~a@ie@bSkfAbr@oOnMwhAzl@u~@im...,4.0,NaN,NaN,4.0,NaN,POINT (-0.16288 49.01589)
34457,1405692,48.302289,-3.895241,29115,zy~Vu}weHqLnp@kXd]gGtHeWp~@a]e@at@`Kyx@hPm_@cY...,NaN,1.0,NaN,NaN,NaN,POINT (-3.89524 48.30229)
34458,366442,47.370808,-0.496845,49308,n|}Aoea`Hud@uJcPoQw@kVlR{Qda@aK`t@oj@~l@{Itj@g...,5.0,4.0,NaN,1.0,NaN,POINT (-0.49684 47.37081)


Jointure with others features for communes :

In [30]:
# Mika a récupéré les geojson des communes dans car_vizs et get-geo-data. Apparemment c'est lourd.



data_path = 'datas'
file_name2 = 'communes-20220101'
ext2 = '.shp'
url ='https://www.data.gouv.fr/fr/datasets/r/0e117c06-248f-45e5-8945-0e79d9136165'
temp_path = 'temp_unzip'


try: 
    communes = gpd.read_feather(os.path.join(data_path,file_name2 + '.feather'))
except:
    # Zip file from url  
    zip_file = requests.get(url).content
    os.makedirs(temp_path, exist_ok=True)
    with zipfile.ZipFile(io.BytesIO(zip_file)) as archive:
        archive.extractall(temp_path)
    communes = gpd.read_file(os.path.join(temp_path,file_name2 + ext2))   
    shutil.rmtree(temp_path) 
    
    communes.to_feather(os.path.join(data_path,file_name2 + '.feather'))
    

In [34]:
file_name = 'voitures-par-commune-par-energie'
communes = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))

--------------------------

Fonctions de requêtes neo4j :

In [5]:
def delete_all(tx):
    query = 'MATCH (n) \
                DETACH DELETE n \
                RETURN count(*) AS COUNT'
    result = tx.run(query)
    return result.data()

In [22]:
def delete_city(tx):
    query = 'MATCH (n:CITY) \
                DETACH DELETE n \
                RETURN count(*) AS COUNT'
    result = tx.run(query)
    return result.data()

In [56]:
def delete_just_edges(tx):
    query = 'MATCH ()-[r]-() \
                DELETE r \
                RETURN count(*) AS COUNT'
    result = tx.run(query)
    return result.data()

In [6]:
def create_node(tx, node):
    query = "   WITH $node AS node \
                CREATE (r:ROAD {road_index:node}) \
                RETURN count(r) AS COUNT"
    
    result = tx.run(query, node=node)
    return result.data()

In [20]:
def create_node_per_batch(tx, props_list):

    query = "   UNWIND $props_list AS map \
                CREATE (n:CITY) \
                SET n = map \
                RETURN count(n) AS COUNT"

    result = tx.run(query, props_list=props_list)
    return result.data()

In [8]:
def create_edge_between_ROAD_POINTS(tx, props_list):

    query = "   UNWIND $props_list AS map \
                MERGE (n1:ROAD_POINT {osmid : map.u})-[r:CONNECT]->(n2:ROAD_POINT {osmid : map.v}) \
                SET r = map \
                RETURN count(r) AS COUNT"

    result = tx.run(query, props_list=props_list)
    return result.data()

In [9]:
def check_collection(tx, id):

    query = "  WITH $id AS id \
                MATCH (n) \
                WHERE n.osmid = id \
                RETURN count(n) \
                 "
    
    result = tx.run(query, id=id)
    return result.data()

In [10]:
def all_fusion_nodes(tx, id):

    query = "   WITH $id AS id \
                MATCH (n) \
                WHERE n.osmid = id \
                WITH collect(n) AS nodes \
                CALL apoc.refactor.mergeNodes( \
                                        nodes, \
                                        {properties: { \
                                        osmid: 'discard', \
                                        `.*`:'combine'} \
                                            }) \
                YIELD node \
                RETURN node "
    
    result = tx.run(query,  id=id)
    return result.data()

In [11]:
def match_alone_nodes(tx):
    query = " MATCH (r) \
            WHERE NOT (r)-[:LINKED_TO]-() \
            RETURN r.road_index AS road_index"
    
    result = tx.run(query)
    return result.data()

Drivers session for Neo4j :

In [12]:
driver = GraphDatabase.driver(
  "bolt://localhost:7687",
  auth=basic_auth("neo4j", PASSWORD_NEO4J))

Ardoise (pour effacer tout ancien graph)

In [23]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(delete_city)

driver.close()

nb_nodes = result[0]['COUNT']
                
print(f'Total Numbers of deleted nodes: {nb_nodes}')

Total Numbers of deleted nodes: 34460


In [40]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(delete_all)

driver.close()

nb_nodes = result[0]['COUNT']
                
print(f'Total Numbers of deleted nodes: {nb_nodes}')

Total Numbers of deleted nodes: 2


Ardoise (pour effacer seulement les relations)

In [158]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(delete_just_edges)

driver.close()

nb_edges = result[0]['COUNT']
                
print(f'Total Numbers of deleted edges: {nb_edges}')

Total Numbers of deleted edges: 0


---

NODES GENERATION :

In [13]:
nodes_reset.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 34460 entries, 0 to 34459
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   osmid     34460 non-null  object  
 1   y         34460 non-null  float64 
 2   x         34460 non-null  float64 
 3   codgeo    34460 non-null  object  
 4   area      34460 non-null  object  
 5   sleeping  15412 non-null  float64 
 6   visit     17921 non-null  float64 
 7   working   6950 non-null   float64 
 8   fooding   6616 non-null   float64 
 9   vehicule  4166 non-null   float64 
 10  geometry  34460 non-null  geometry
dtypes: float64(7), geometry(1), object(3)
memory usage: 2.9+ MB


In [14]:
nodes_reset.drop(['area'], axis=1, inplace=True)

In [15]:
primary_key = 'osmid'
object_entity = 'COMMUNE'

In [16]:
nodes_reset.drop(['geometry'], axis=1, inplace=True)

In [17]:
print("conversion of objects to parameters list for neo4j...")

props_list = nodes_reset.to_dict("records")

conversion of objects to parameters list for neo4j...


In [18]:
BATCH_SIZE = 10000

In [19]:
# --- Batch function ---

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]


In [21]:
with driver.session() as session:

    count_list = []

    # Batch the writing session :

    for props_batch in batch(props_list, BATCH_SIZE):
        result = session.write_transaction(create_node_per_batch, props_batch)
        count_list.append(result[0]['COUNT'])
        nb_updates = sum(count_list)
        print(f'Number of nodes created : {nb_updates}...')

        

    driver.close()

print(f'Bath_list: {count_list}')

print(f'Total Numbers of nodes created/updates: {nb_updates}')

/tmp/ipykernel_20387/3878216737.py:8: DeprecationWarning: write_transaction has been renamed to execute_write
  result = session.write_transaction(create_node_per_batch, props_batch)


Number of nodes created : 10000...
Number of nodes created : 20000...
Number of nodes created : 30000...
Number of nodes created : 34460...
Bath_list: [10000, 10000, 10000, 4460]
Total Numbers of nodes created/updates: 34460


---